In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Importing the dataset
train = pd.read_csv('dataset/train.csv',header=0)
test = pd.read_csv('dataset/test.csv',header=0)

train['isTrain'] = 1
test['isTrain'] = 0
test['P'] = np.nan
dataset = train.append(test, ignore_index=True)


In [2]:
# dataset = dataset.apply(lambda x:x.fillna(x.value_counts().index[0]))
# dataset.head()

In [3]:
categorical = ['A', 'D', 'E', 'F', 'G', 'I', 'J', 'L', 'M']
for cols in categorical:
#     pd.get_dummies(dataset[col],prefix=col, drop_first=True)
    dataset = pd.concat([dataset,pd.get_dummies(dataset[cols], prefix=cols)],axis=1)
    dataset.drop([cols],axis=1, inplace=True)

dataset.head()

,B,C,H,K,N,O,P,id,isTrain,A_a,...,G_z,I_f,I_t,J_f,J_t,L_f,L_t,M_g,M_p,M_s
0,18.42,10.415,0.125,0,120.0,375,1.0,1,1,0,...,0,0,1,1,0,1,0,1,0,0
1,21.75,11.750,0.250,0,180.0,0,1.0,2,1,1,...,0,1,0,1,0,0,1,1,0,0
2,30.17,1.085,0.040,0,170.0,179,1.0,3,1,0,...,0,1,0,1,0,1,0,1,0,0
3,22.67,2.540,2.585,0,0.0,0,0.0,4,1,0,...,0,0,1,1,0,1,0,1,0,0
4,36.00,1.000,2.000,11,0.0,456,0.0,5,1,1,...,0,0,1,0,1,1,0,1,0,0


In [4]:
column_names = dataset.columns.values

X = dataset[['B', 'C', 'H', 'K', 'N', 'O', 'A_a', 'A_b', 'D_l',
       'D_u', 'D_y', 'E_g', 'E_gg', 'E_p', 'F_aa', 'F_c', 'F_cc', 'F_d',
       'F_e', 'F_ff', 'F_i', 'F_j', 'F_k', 'F_m', 'F_q', 'F_r', 'F_w',
       'F_x', 'G_bb', 'G_dd', 'G_ff', 'G_h', 'G_j', 'G_n', 'G_o', 'G_v',
       'G_z', 'I_f', 'I_t', 'J_f', 'J_t', 'L_f', 'L_t', 'M_g', 'M_p',
       'M_s','isTrain']]
y = dataset[['P', 'isTrain']]

In [5]:
X_train = X[X['isTrain'] ==1]
X_test = X[X['isTrain'] ==0]
y_train = y[y['isTrain']==1]
X_train.drop(['isTrain'],axis=1, inplace=True)
X_test.drop(['isTrain'],axis=1, inplace=True)
y_train.drop(['isTrain'],axis=1, inplace=True)

/home/entrophy/.local/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/entrophy/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/entrophy/.local/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
def modelfit(alg, X_train, y_train, X_test, test, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(X_train.values, label=y_train.values)
        xgtest = xgb.DMatrix(X_test.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, verbose_eval=True,)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(X_train, y_train,eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(X_train)
    test['P'] = alg.predict(X_test)
    test.P = test.P.astype(int)
    submission = test[['id', 'P']]
    submission.to_csv("sub_xgboost.csv", index=False)
    dtrain_predprob = alg.predict_proba(X_train)[:,1]
        
    #Print model report:
    print "\nModel Report"
    print "Accuracy : %.4g" % metrics.accuracy_score(y_train.values, dtrain_predictions)
    print "AUC Score (Train): %f" % metrics.roc_auc_score(y_train, dtrain_predprob)
    


In [7]:
#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search


xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        seed=27)
modelfit(xgb1, X_train, y_train, X_test, test )  

[0]	train-auc:0.937909+0.0107452	test-auc:0.908272+0.0247213
[1]	train-auc:0.953668+0.00796059	test-auc:0.916306+0.0207919
[2]	train-auc:0.963515+0.0051535	test-auc:0.927434+0.0210651
[3]	train-auc:0.96613+0.00467967	test-auc:0.931305+0.0197916
[4]	train-auc:0.96912+0.00627732	test-auc:0.928835+0.0224145
[5]	train-auc:0.971653+0.00420485	test-auc:0.930251+0.024406
[6]	train-auc:0.973707+0.00345264	test-auc:0.93266+0.0243978
[7]	train-auc:0.97429+0.00349047	test-auc:0.932325+0.0248623
[8]	train-auc:0.97657+0.00339493	test-auc:0.935842+0.0228948
[9]	train-auc:0.978668+0.00304226	test-auc:0.934993+0.0212413


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


[10]	train-auc:0.979023+0.0029093	test-auc:0.936199+0.0214499
[11]	train-auc:0.979967+0.00295865	test-auc:0.936349+0.0217338
[12]	train-auc:0.980328+0.00260553	test-auc:0.935153+0.0214751
[13]	train-auc:0.981579+0.0025676	test-auc:0.935237+0.0217327
[14]	train-auc:0.982524+0.00213865	test-auc:0.934646+0.021263
[15]	train-auc:0.983493+0.00215886	test-auc:0.93496+0.020958
[16]	train-auc:0.983705+0.00242005	test-auc:0.935607+0.020161
[17]	train-auc:0.984907+0.00198967	test-auc:0.935595+0.0202704
[18]	train-auc:0.985457+0.0016867	test-auc:0.937574+0.0198054
[19]	train-auc:0.98592+0.00165524	test-auc:0.937793+0.0200953
[20]	train-auc:0.986451+0.00138493	test-auc:0.937471+0.0196829
[21]	train-auc:0.986866+0.00143079	test-auc:0.937181+0.0208569
[22]	train-auc:0.987467+0.00175167	test-auc:0.936392+0.0202355
[23]	train-auc:0.988115+0.00148706	test-auc:0.936781+0.0209687
[24]	train-auc:0.988556+0.00139773	test-auc:0.936775+0.0209445
[25]	train-auc:0.98911+0.00118992	test-auc:0.936862+0.0213275
[

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an err


Model Report
Accuracy : 0.9656
AUC Score (Train): 0.993416


In [ ]:
#Grid seach on subsample and max_features
#Choose all predictors except target & IDcols
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=1000, max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train.iloc[:,:],y_train.iloc[:,0])

In [ ]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_